In [1]:
import os
os.chdir('../src/gpu_spatial_graph_pipeline')

%load_ext autoreload
%autoreload 2

In [2]:
os.getcwd()

'/home/marcella/Documents/gpu-spatial-graph-pipeline/src/gpu_spatial_graph_pipeline'

In [16]:
import os
from argparse import ArgumentParser, Namespace
from random import choices
import pytorch_lightning as pl
from typing import Callable, List, Optional, Sequence, Union
import squidpy as sq
import torch
from torch_geometric.loader import RandomNodeSampler
import pandas as pd
from torch_geometric.data import Data
from anndata import AnnData
from utils import adata2data, to_one_data
from data.datamodule import GraphAnnDataModule
from models.linear_ncem import LinearNCEM

In [4]:
#Mibitof
adata = sq.datasets.mibitof()
#feature_name=adata.obs.keys()[0] #Use for IMC dataset

#specify features to use
feature_names=['Cluster','batch']

#input of datamodule
num_features=(len(set(adata.obs[feature_names[0]])),len(set(adata.obs[feature_names[1]])))
num_genes=adata.X.shape[1]

# #IMC
# adata = sq.datasets.imc()

# #specify features to use
# feature_names=['cell type']

# #input of datamodule
# num_features=len(set(adata.obs[feature_names[0]]))
# num_genes=adata.X.shape[1]


In [44]:
dm = GraphAnnDataModule(adata=adata, feature_names=feature_names, adata2data_fn=to_one_data, num_workers = 8, batch_size=1)

In [48]:
dm.setup()

In [49]:
for batch in dm.train_dataloader():
    print(batch)

Data(num_nodes=1023, edge_index=[2, 6138], x=[1023, 11], y=[1023, 36])


In [50]:
model = LinearNCEM(in_channels=num_features,out_channels=num_genes, model_type='spatial', lr=0.0001,weight_decay=0.000001)

In [38]:
gpu=False
if gpu:
    trainer:pl.Trainer = pl.Trainer(accelerator='gpu',max_epochs=100,log_every_n_steps=1)
else:
    trainer:pl.Trainer = pl.Trainer(accelerator='cpu',max_epochs=100,log_every_n_steps=1)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [39]:
trainer.fit(model,datamodule=dm)


  | Name        | Type            | Params
------------------------------------------------
0 | model_sigma | LinearSpatial   | 2.7 K 
1 | model_mu    | LinearSpatial   | 2.7 K 
2 | loss_module | GaussianNLLLoss | 0     
------------------------------------------------
5.5 K     Trainable params
0         Non-trainable params
5.5 K     Total params
0.022     Total estimated model params size (MB)


/home/marcella/miniconda3/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:98: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 2. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Epoch 27:   0%|          | 0/2 [00:00<?, ?it/s, loss=0.00325, v_num=1, val_r2_score=-1.11, val_loss=-.000358]        

Exception ignored in: <function _releaseLock at 0x7ff8b8eeff40>
Traceback (most recent call last):
  File "/home/marcella/miniconda3/envs/gnn/lib/python3.10/logging/__init__.py", line 228, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 


Epoch 80:   0%|          | 0/2 [00:00<?, ?it/s, loss=-0.0147, v_num=1, val_r2_score=-.797, val_loss=-.0182]           

Exception ignored in: <function _releaseLock at 0x7ff8b8eeff40>
Traceback (most recent call last):
  File "/home/marcella/miniconda3/envs/gnn/lib/python3.10/logging/__init__.py", line 228, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 


Epoch 99: 100%|██████████| 2/2 [00:00<00:00,  5.53it/s, loss=-0.0214, v_num=1, val_r2_score=-.699, val_loss=-.0248]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 2/2 [00:00<00:00,  5.45it/s, loss=-0.0214, v_num=1, val_r2_score=-.699, val_loss=-.0248]
